<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/generation_of_Llama3_1_8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# stting up

In [2]:
!pip install transformers
!pip install huggingfacehub
!pip install torch
!pip install accelerate

ERROR: Could not find a version that satisfies the requirement huggingfacehub (from versions: none)
ERROR: No matching distribution found for huggingfacehub


In [3]:
import os
import torch
import json
import pandas as pd
from google.colab import files
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
from IPython.display import FileLink, display


os.environ['HF_TOKEN']= ''
os.environ['HUGGINGFACEHUB_API_TOKEN']= ''

 **loading the model**

In [4]:
# Load the model and tokenizer
model_id = "meta-llama/Meta-Llama-3.1-8B"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Create the text generation pipeline
pipeline = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer
)

# Test the pipeline
print(pipeline("How old are you?"))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'How old are you? Do you know what you want to do with your life? Do you have a plan to get there'}]


# Generates multiple answers for a given question using the model pipeline


In [5]:
def generate_answer(question, num_samples):
    prompt = f" أجب على السؤال التالي بجملة واحدة موجزة ولكن كاملة باللغة العربية.\nQuestion: {question}\nAnswer:"

    # Generate the first 9 answers with temperature=1.0
    outputs_high_temp = pipeline(
        prompt,
        max_length=100,
        truncation=True,
        temperature=1.0,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=num_samples - 1
    )

    # Generate the last answer
    outputs_low_temp = pipeline(
        prompt,
        max_length=100,
        truncation=True,
        temperature=0.1,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=1
    )

    # Combine the results
    responses = [
        output['generated_text'].split("Answer:")[-1].strip() for output in outputs_high_temp
    ] + [
        outputs_low_temp[0]['generated_text'].split("Answer:")[-1].strip()
    ]

    return responses


# main function to to load questions and gerate answers and save them to a file

In [7]:
# Main function
def main():
    # Step 1: Upload and read the file
    file_path = '/content/train-open.json'
    if os.path.exists(file_path):
        print("File found")
    else:
        print("File not found")
        return

    with open(file_path, 'r', encoding='utf-8') as file:
          data = json.load(file)

    # Convert to DataFrame
    df = pd.DataFrame(data)

    # Initialize an empty list to store the results
    results = []

    # Step 2:  Generate answers for a sample of questions
    for index, row in df.sample(n=50).iterrows() :
        question = row['question']
        original_answer = row.get('answer', "")  # Use empty string if no original answer provided
        question_id = row.get('question_id', index)  # Use index if no question_id provided

        # Generate 10 answers for the 50 questions
        generated_answers = generate_answer(question=question, num_samples=10)

        # Append the results
        results.append({
            "question_id": question_id,
            "question": question,
            "original_answer": original_answer,
            "generated_answers": generated_answers
        })

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)
    print(results_df)

    filename = "generated_answers.json"
    # Step 3: Save results to a JSON file
    with open(filename, "w", encoding="utf-8") as json_file:
          json.dump(results, json_file, ensure_ascii=False, indent=4)

    print("Generated answers saved to 'generated_answers.json'")
    display(FileLink(filename))
    files.download("generated_answers.json")

    #what i think next stpes is
    # Step 4: Clustering
    # Step 5: Calculate probability
    # Step 6: Calculate semantic entropy
    # Step 7: show results

# Run the main function
if __name__ == "__main__":
    main()

File found


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_to

   question_id                                           question  \
0      1118818                      متي توفي توماس هربرت ماكنتوش؟   
1      1115586                   ما هي نلطحة السحاب أستراليا 108؟   
2      1143205  ما هي العلاقة بين الوهابيين والمملكة العثمانية...   
3      1002207  ما هي الرها وما دورها في الحملة الصليبية الأول...   
4      1125540                                   أين توفي سوبيا ؟   
5      1164816   من المسؤولين عن تطوير نظرية "عدد ضحايا كلينتون"؟   
6      1074005                         ما هو محتوى فيلم "Zarê"؟\n   
7      1167486                       من هم أبطال فيلم حالة العنف؟   
8      1172077        ما هو مركز لعب كريستوف سوتر في كرة القدم؟\n   
9      1188648   ما هو مركز لعب لوكا-ميلان تساندر في كرة القدم؟\n   
10     1164088  ما هي الخلافات التي نشأت بين البهائيين العباسي...   
11     1185184                         متى تم اكتشاف حقل إريدو؟\n   
12     1158233                   ما هو السياق الأوسع للمعركة؟\n\n   
13     1073990  ما هي المشكلة المر

/content/generated_answers.json

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>